In [ ]:
%%configure -f
{
  "conf": {
    "spark.notebook.parameters": "{}"
  },
  "defaultLakehouse": {
    "name": "BenchmarkLakehouse"
  }
}


# 📓 Benchmark Reset Helper Notebook

Use this notebook to reset synthetic data files and drop all tables from BenchmarkLakehouse and BenchmarkWarehouse in Microsoft Fabric.
- **Cell 1:** Delete all synthetic data files from DataSourceLakehouse
- **Cell 2:** Delete all tables from BenchmarkLakehouse
- **Cell 3:** Delete all tables from BenchmarkWarehouse

**WARNING:** All operations are destructive! Run only what you need.

## 1. Reset Synthetic Data (Lakehouse Files)
Deletes all files and folders in `DataSourceLakehouse/Files/base/` and `DataSourceLakehouse/Files/updates/` using abfss paths.

In [ ]:
# WARNING: This deletes ALL synthetic data files from DataSourceLakehouse
from notebookutils import fs

folders = [
    "abfss://FabricBenchmarking@onelake.dfs.fabric.microsoft.com/DataSourceLakehouse.Lakehouse/Files/base/",
    "abfss://FabricBenchmarking@onelake.dfs.fabric.microsoft.com/DataSourceLakehouse.Lakehouse/Files/updates/"
]

for folder in folders:
    try:
        fs.rm(folder, True)  # True = recursive delete
        print(f"Deleted folder and all contents: {folder}")
    except Exception as e:
        print(f"An error occurred deleting {folder}: {e}")

## 2. Reset Ingestion — Drop All Tables from BenchmarkLakehouse
Drops all tables in the Lakehouse using Spark SQL.

In [ ]:
# WARNING: This drops ALL tables from BenchmarkLakehouse
try:
    tables = spark.catalog.listTables('BenchmarkLakehouse')
    for t in tables:
        spark.sql(f"DROP TABLE IF EXISTS BenchmarkLakehouse.{t.name}")
    print('All tables dropped from BenchmarkLakehouse.')
except Exception as e:
    print('Error during BenchmarkLakehouse table reset:', e)

## 3. Reset Ingestion — Drop All Tables from BenchmarkWarehouse
Drops all tables in the Warehouse using T-SQL. **This cell must be run in a BenchmarkWarehouse SQL Query, not in the notebook.**

In [ ]:
-- WARNING: This drops ALL tables from BenchmarkWarehouse
-- THIS DOES NOT WORK HERE.  
-- Run it in a BenchmarkWarehouse SQL Query

-- List all tables in dbo schema
SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo';

-- Bulk drop all tables in dbo schema
DECLARE @sql NVARCHAR(MAX) = N'';
SELECT @sql += 'DROP TABLE IF EXISTS [dbo].[' + TABLE_NAME + '];'
FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo';
EXEC sp_executesql @sql;

---
Run individual cells as needed. Operations are destructive!